In [1]:
!pwd

/home/nicksung/Desktop/nicksung/bwb_full_v2


In [2]:
#!/usr/bin/env python3
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import vtk

from dataset_v2 import UnifiedDesignDataset, design_collate_fn
from film_model_v1 import FiLMNet

# ——— Paths ———
trainval_csv     = "./processed/final_data_train_val.csv"
trainval_pts_h5  = "/home/nicksung/Desktop/nicksung/bwb_full/surface_point/surface_data_train_val.hdf5"
trainval_norm_h5 = "/home/nicksung/Desktop/nicksung/bwb_full/surface_point/normals_data_train_val.hdf5"

test_csv         = "/home/nicksung/Desktop/nicksung/bwb_full/processed/final_data_test.csv"
test_pts_h5      = "/home/nicksung/Desktop/nicksung/bwb_full/surface_point/surface_data_test.hdf5"
test_norm_h5     = "/home/nicksung/Desktop/nicksung/bwb_full/surface_point/normals_data_test.hdf5"

model_path       = "./film_model_saved_weights/final_model_v1.pth"
bestpred_npy     = "/home/nicksung/Desktop/nicksung/bwb_full/point_to_parameter/best_predictions.npy"

original_vtk_dir = "/home/nicksung/Desktop/nicksung/bwb_full/data_test"
vtk_out_dir      = "./vtk_pred_param"
os.makedirs(vtk_out_dir, exist_ok=True)

batch_size = 64
device     = "cuda" if torch.cuda.is_available() else "cpu"

# ——— Load best‐predicted shape parameters ———
bestpred_array = np.load(bestpred_npy)  # shape (N_geoms, 9)

# -----------------------------------------------------------------------------
# Unnormalize helper
# -----------------------------------------------------------------------------
def unnormalize_coeffs(coeffs_norm, dataset):
    cp  = coeffs_norm[:,0]*dataset.output_std[0] + dataset.output_mean[0]
    cfx = coeffs_norm[:,1]*dataset.output_std[1] + dataset.output_mean[1]
    cfz = coeffs_norm[:,2]*dataset.output_std[2] + dataset.output_mean[2]
    return np.column_stack([cp,cfx,cfz])

# -----------------------------------------------------------------------------
# Metric computation
# -----------------------------------------------------------------------------
def compute_test_metrics_from_loader(model, loader, dataset, device='cuda'):
    model.eval()
    sse = np.zeros(3); sae = np.zeros(3)
    sum_gt_abs = np.zeros(3); sum_gt_sq = np.zeros(3)
    total=0
    with torch.no_grad():
        for coords,conds,targets in loader:
            coords,conds,targets = coords.float().to(device), conds.float().to(device), targets.float().to(device)
            preds = model(coords,conds)
            p_np, t_np = preds.cpu().numpy(), targets.cpu().numpy()
            p_u = unnormalize_coeffs(p_np, dataset)
            t_u = unnormalize_coeffs(t_np, dataset)
            diff = p_u - t_u
            sse        += np.sum(diff**2,axis=0)
            sae        += np.sum(np.abs(diff),axis=0)
            sum_gt_abs += np.sum(np.abs(t_u),axis=0)
            sum_gt_sq  += np.sum(t_u**2,    axis=0)
            total     += t_u.shape[0]
    mse    = sse/total; mae = sae/total
    eps = 1e-20
    rel_l1 = sae/(sum_gt_abs+eps); rel_l2 = sse/(sum_gt_sq+eps)
    keys=["cp","cf_x","cf_z"]
    return dict(zip(keys,mse)), dict(zip(keys,mae)), dict(zip(keys,rel_l1)), dict(zip(keys,rel_l2))

# -----------------------------------------------------------------------------
# Model loader
# -----------------------------------------------------------------------------
def load_trained_model(model_path, device='cuda'):
    model = FiLMNet(13,6,3,256,4,3)
    sd = torch.load(model_path,map_location=device)
    model.load_state_dict(sd)
    model.to(device).eval()
    return model

# -----------------------------------------------------------------------------
# VTK I/O
# -----------------------------------------------------------------------------
def read_original_mesh(fn):
    if not os.path.isfile(fn): return None,None
    r=vtk.vtkPolyDataReader(); r.SetFileName(fn); r.Update()
    pd_ = r.GetOutput()
    if pd_.GetNumberOfPoints()==0: return None,None
    pts = np.array([pd_.GetPoint(i) for i in range(pd_.GetNumberOfPoints())],dtype=np.float32)
    polys = pd_.GetPolys()
    return pts, polys

def write_vtk_with_mesh(coords,polys,gt,true_p,best_p,fn):
    n=coords.shape[0]
    pts=vtk.vtkPoints(); pts.SetNumberOfPoints(n)
    for i,(x,y,z) in enumerate(coords): pts.SetPoint(i,float(x),float(y),float(z))
    pd_ = vtk.vtkPolyData(); pd_.SetPoints(pts); pd_.SetPolys(polys)
    for name,idx in zip(["cp","cf_x","cf_z"], range(3)):
        gt_arr   = gt[:,idx]
        t_arr    = true_p[:,idx]
        b_arr    = best_p[:,idx]
        err_t    = np.abs(gt_arr - t_arr)
        err_b    = np.abs(gt_arr - b_arr)
        for arr,sfx in [(gt_arr,"ground_truth"),
                        (t_arr,"pred_true"),
                        (err_t,"err_true"),
                        (b_arr,"pred_best"),
                        (err_b,"err_best")]:
            fa=vtk.vtkFloatArray(); fa.SetName(f"{name}_{sfx}")
            fa.SetNumberOfComponents(1); fa.SetNumberOfValues(n)
            for j,v in enumerate(arr): fa.SetValue(j,float(v))
            pd_.GetPointData().AddArray(fa)
    w=vtk.vtkPolyDataWriter(); w.SetFileName(fn); w.SetInputData(pd_); w.Write()
    print(f"Wrote {fn}")

# -----------------------------------------------------------------------------
# Main
# -----------------------------------------------------------------------------
if __name__=="__main__":
    # 1) Norm stats from train+val
    trainval_ds = UnifiedDesignDataset(trainval_csv,
                                       trainval_pts_h5,
                                       trainval_norm_h5,
                                       mode="train")
    norm_stats = trainval_ds.norm_stats
    print("Norm stats:",norm_stats)

    # 2) Test set
    test_ds = UnifiedDesignDataset(test_csv,
                                   test_pts_h5,
                                   test_norm_h5,
                                   norm_stats=norm_stats,
                                   mode="test")
    print("Test designs:",len(test_ds))

    # 3) Metrics TRUE shape
    loader = DataLoader(test_ds,batch_size=batch_size,
                        collate_fn=design_collate_fn,shuffle=False)
    model  = load_trained_model(model_path,device=device)
    mse_t,mae_t,r1_t,r2_t = compute_test_metrics_from_loader(model,loader,test_ds,device)
    print("\n=== TRUE shape metrics ===")
    for d in ["cp","cf_x","cf_z"]:
        print(f"{d}: MSE={mse_t[d]:.6e}, MAE={mae_t[d]:.6e}, RelL1={r1_t[d]:.6e}, RelL2={r2_t[d]:.6e}")

    # 4) Inject bestpred into each design_item via enumerate → geom_idx
    for geom_idx, design_item in enumerate(test_ds.design_info):
        for case in design_item:
            # stash original
            case['flight_cond_true'] = case['flight_cond'].copy()
            # pull bestpred
            shape_pred = bestpred_array[geom_idx]  # safe lookup 0…N-1
            # normalize shape_pred
            shp_n = (shape_pred - trainval_ds.shape_mean)/trainval_ds.shape_std
            fl_np = case['flight_cond_true'][:4]
            case['flight_cond'] = np.concatenate([fl_np, shp_n],axis=0)

    # 5) Metrics BEST shape
    mse_b, mae_b, r1_b, r2_b = compute_test_metrics_from_loader(model,loader,test_ds,device)
    print("\n=== BEST shape metrics ===")
    for d in ["cp","cf_x","cf_z"]:
        print(f"{d}: MSE={mse_b[d]:.6e}, MAE={mae_b[d]:.6e}, RelL1={r1_b[d]:.6e}, RelL2={r2_b[d]:.6e}")

    # 6) Export comparison VTKs
    for geom_idx, design_item in enumerate(test_ds.design_info):
        for case in design_item:
            pts6   = case['points']
            tgt    = case['coeffs']
            flt_t  = case['flight_cond_true']
            flt_b  = case['flight_cond']
            N = pts6.shape[0]
            coords = pts6[:,:3]   # mesh coords
            normals= pts6[:,3:]   # normals (not used in mesh write)
            # predict true
            ct = torch.from_numpy(flt_t).float().unsqueeze(0).expand(N,-1).to(device)
            x  = torch.from_numpy(pts6).float().to(device)
            with torch.no_grad(): p_t = model(x,ct).cpu().numpy()
            # predict best
            cb = torch.from_numpy(flt_b).float().unsqueeze(0).expand(N,-1).to(device)
            with torch.no_grad(): p_b = model(x,cb).cpu().numpy()
            # unnorm
            gt_u   = unnormalize_coeffs(case['coeffs'],test_ds)
            t_u    = unnormalize_coeffs(p_t,       test_ds)
            b_u    = unnormalize_coeffs(p_b,       test_ds)
            # mesh io
            mesh_fn = os.path.join(original_vtk_dir,f"{case['case_name']}.vtk")
            mpts,polys = read_original_mesh(mesh_fn)
            if mpts is None: continue
            out_fn = os.path.join(vtk_out_dir, f"{case['case_name']}_cmp.vtk")
            write_vtk_with_mesh(mpts,polys,gt_u,t_u,b_u,out_fn)

    print("\nDone. Comparison VTKs in", vtk_out_dir)


Unique geometries in CSV: 999
Unique geometries in dataset after HDF5 filtering: 999
Norm stats: {'shape_mean': array([149.73118913, 124.91552208, 445.6917667 , 699.66621744,
       230.41100793,  75.01262061,  49.9939966 ,  49.98475764,
        31.92120612]), 'shape_std': array([ 28.91160165,  43.30243653, 144.01147529,  86.77421237,
        28.92184576,   8.65547754,   5.77840385,   5.78463163,
         4.61681044]), 'flight_mean': array([2.02570334e+01, 1.74719006e+07, 2.66649940e-01, 4.93905060e+00]), 'flight_std': array([1.15438056e+01, 1.61492810e+07, 1.29571841e-01, 8.63705480e+00]), 'coord_min': array([-7.14891175e-17, -1.05967498e+00, -9.10214633e-02]), 'coord_max': array([1.16249204, 1.05967498, 0.09102037]), 'output_mean': array([-0.10253485,  0.0074317 ,  0.00028973]), 'output_std': array([0.45806409, 0.02630924, 0.01266003])}
Unique geometries in CSV: 100
Unique geometries in dataset after HDF5 filtering: 100
Test designs: 100

=== TRUE shape metrics ===
cp: MSE=7.867552e-

In [3]:
# 6) Find lowest-error cases (no VTK export)
case_errors = []

for geom_idx, design_item in enumerate(test_ds.design_info):
    for case in design_item:
        pts6   = case['points']
        tgt    = case['coeffs']
        flt_b  = case['flight_cond']  # injected bestpred
        N      = pts6.shape[0]

        # Model prediction with best-predicted shape
        cb = torch.from_numpy(flt_b).float().unsqueeze(0).expand(N, -1).to(device)
        x  = torch.from_numpy(pts6).float().to(device)
        with torch.no_grad(): p_b = model(x, cb).cpu().numpy()

        # Unnormalize predictions and ground truth
        gt_u = unnormalize_coeffs(tgt, test_ds)
        b_u  = unnormalize_coeffs(p_b, test_ds)

        # Compute total L1 error over all points and all 3 coefficients
        err = np.abs(gt_u - b_u)
        total_err = err.mean()
        case_errors.append((case['case_name'], total_err))

# Sort by error and show top 10
case_errors.sort(key=lambda x: x[1])
print("\n=== Top 10 lowest-error cases ===")
for name, err in case_errors[:10]:
    print(f"{name}: Total L1 Error = {err:.6f}")



=== Top 10 lowest-error cases ===
case_717: Total L1 Error = 0.004688
case_149: Total L1 Error = 0.004694
case_705: Total L1 Error = 0.004782
case_143: Total L1 Error = 0.004785
case_653: Total L1 Error = 0.004804
case_614: Total L1 Error = 0.004965
case_562: Total L1 Error = 0.005025
case_500: Total L1 Error = 0.005049
case_881: Total L1 Error = 0.005117
case_459: Total L1 Error = 0.005143
